<a href="https://colab.research.google.com/github/CSTOMJason/nlp/blob/master/lstm_email.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#这是kaggle上的一个数据集上面是使用的keras实现这儿我用tensorflow实现
#注意模型的效果还受到初始化的影响你可以尝试不同的初始化来实验模型的效果
!pip install sklearn 


In [2]:
#加载所用的库
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
%matplotlib inline

Using TensorFlow backend.


In [3]:
#使用pandas读取数据
df = pd.read_csv('spam.csv',delimiter=',',encoding='latin-1')
df.head()#查看前面的数据

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [4]:
#去掉多多余项
df.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'],axis=1,inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
v1    5572 non-null object
v2    5572 non-null object
dtypes: object(2)
memory usage: 87.1+ KB


In [0]:
X = df.v2
Y = df.v1
le = LabelEncoder()
Y = le.fit_transform(Y)
Y = Y.reshape(-1,1)

In [0]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.15)#训练集和测试集的分割

In [0]:
#将训练和测试的数据转换为vector（词序列--我差点坑在这而还好有个it前辈的指导)
max_words = 1000
max_len = 150
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)
sequences_matrix=sequences_matrix
trainx=sequences_matrix
trainy=Y_train
#处理test数据同训练的数据
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)
test_sequences_matrix=test_sequences_matrix
testx=test_sequences_matrix
testy=Y_test


In [0]:
#make标签为one_hot格式
def one_HOT(label):
  l=[]
  for e in label:
    e=list(e)
    if 0 in e:
      e.append(1)
    else:
      e.append(0)
    l.append(e)
  return np.array(l)
    

In [0]:
trainy=one_HOT(trainy)
testy=one_HOT(testy)

In [0]:
import tensorflow as tf
xs=tf.placeholder(tf.int32,[None,trainx.shape[1]])
ys=tf.placeholder(tf.float32,[None,2])
batch_size=tf.placeholder(tf.int32,[])
lr=0.0001
hidden=512

In [0]:
def RNN(x,weight,baise):
  #x=tf.reshape(x,[-1,10,15])
  lstm=lstm_cell=tf.contrib.rnn.BasicLSTMCell(hidden,forget_bias=True,state_is_tuple=True)
  init=lstm.zero_state(batch_size,dtype=tf.float32)
  outputs,states=tf.nn.dynamic_rnn(lstm_cell,inputs=x,initial_state=init,time_major=False)
  print(outputs)
  prediction=tf.nn.relu(tf.matmul(states[-1],weight))+biase
  return prediction

In [12]:
weight=tf.Variable(tf.random_uniform([hidden,2]))
biase=tf.Variable(tf.random_uniform([2]))
embedding=tf.Variable(tf.random_uniform([trainx.shape[0],128]))#embedding层
embeded=tf.nn.embedding_lookup(embedding,xs)
pred=RNN(embeded,weight,biase)


Instructions for updating:
This class is deprecated, please use tf.nn.rnn_cell.LSTMCell, which supports all the feature this cell currently has. Please replace the existing code with tf.nn.rnn_cell.LSTMCell(name='basic_lstm_cell').
Tensor("rnn/transpose_1:0", shape=(?, 150, 512), dtype=float32)


In [0]:
cross_entropy=tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits_v2(logits=pred,labels=ys))
train_op=tf.train.AdamOptimizer(lr).minimize(cross_entropy)
correct_pred=tf.equal(tf.argmax(pred,1),tf.argmax(ys,1))
accuracy=tf.reduce_mean(tf.cast(correct_pred,tf.float32))


In [0]:
batch=128
num_batch=trainx.shape[0]//batch


In [17]:
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  for i in range(100):
    for j in range(num_batch):
      batch_x=trainx[j*batch:(j+1)*batch]
      batch_y=trainy[j*batch:(j+1)*batch]
      sess.run(train_op,feed_dict={xs:batch_x,ys:batch_y,batch_size:batch})
    #print(sess.run(pred,feed_dict={xs:batch_x,batch_size:50}))
    #print(sess.run(cross_entropy,feed_dict={xs:testx,ys:testy,batch_size:testx.shape[0]}))
    if i%5==0:
      print("%dth--->the loss is %f in the test data"%(i,sess.run(cross_entropy,feed_dict={xs:testx,ys:testy,batch_size:testx.shape[0]})))
      print("%dth--->the accuracy is %f in the test data"%(i,sess.run(accuracy,feed_dict={xs:testx,ys:testy,batch_size:testx.shape[0]})))

0th--->the loss is 0.295884 in the test data
0th--->the accuracy is 0.899522 in the test data
5th--->the loss is 0.133478 in the test data
5th--->the accuracy is 0.953349 in the test data
10th--->the loss is 0.099922 in the test data
10th--->the accuracy is 0.962919 in the test data
15th--->the loss is 0.093209 in the test data
15th--->the accuracy is 0.967703 in the test data
20th--->the loss is 0.092865 in the test data
20th--->the accuracy is 0.970096 in the test data
25th--->the loss is 0.079977 in the test data
25th--->the accuracy is 0.971292 in the test data
30th--->the loss is 0.087813 in the test data
30th--->the accuracy is 0.974880 in the test data
35th--->the loss is 0.080888 in the test data
35th--->the accuracy is 0.977273 in the test data
40th--->the loss is 0.084392 in the test data
40th--->the accuracy is 0.977273 in the test data
45th--->the loss is 0.093759 in the test data
45th--->the accuracy is 0.977273 in the test data
50th--->the loss is 0.095902 in the test dat